In [1]:
import numpy as np
import sympy as sm
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy import optimize

In this model project we want to solve the Solow model with humancapital. We take the model from 'Introducing Advanced Macroeconomics - Growth and Business Cycles'. 3 edition 2022, Sørensen, Peter Birch and Whitta-Jacobsen, Hans Jørgen.

The Solow model with human capital from Mankiw, Weil and Romer is an extenstion of the general Solow model, that helps explain why international investments does not flow to poorer countries as predicted by the general Solow model. The modification finds itself in the addition of human capital, which is mostly measured as years of education and experience in general, and sheds greater light on why some countries are rich and some are poor. 

# The model

In [2]:
#First we define all the parameters and variables using sympy
Y = sm.symbols('Y_t')           #Output
K = sm.symbols('K_t')           #Physical capital
alpha = sm.symbols('alpha')     #Income share of capital and labour
H = sm.symbols('H_t')           #Human Capital
phi = sm.symbols('phi')         #Income share of Human capital
A = sm.symbols('A_t')           #Technology
L = sm.symbols('L_t')           #Labour
r = sm.symbols('r_t')           #Real rentan rate on capital
w = sm.symbols('w_t')           #Real wage
n = sm.symbols('n')             #Growth rate of Labour
g = sm.symbols('g')             #Growth rate of technology
K1 = sm.symbols('K_{t+1}')      #Capital in the next period
sk = sm.symbols('s_K')          #Savings rate on physical capital
sh = sm.symbols('s_H')          #Savings rate on human capital
H1 = sm.symbols('H_{t+1}')      #Human capital in the next period
delta = sm.symbols('delta')     #Depreciation rate (both on physical and human capital!)
L1 = sm.symbols('L_{t+1}')      #Labour in the next period
A1 = sm.symbols('A_{t+1}')      #Technology in the next period

In [3]:
#Here we create all the equations of the model and displays them again using sympy
Y_t = sm.Eq(Y, (K**alpha*H**phi)*(A*L)**(1-alpha-phi))
r_t = sm.Eq(r, alpha*(K/(A*L))**(alpha-1)*(H/(A*L))**phi)
w_t = sm.Eq(w, (1-alpha)*(K/(A*L))**alpha*(H/(A*L))**phi*A)
K_t1 = sm.Eq(K1, sk*Y+(1-delta)*K)
H_t1 = sm.Eq(H1, sh*Y+(1-delta)*H)
L_t1 = sm.Eq(L1, (1+n)*L)
A_t1 = sm.Eq(A1, (1+g)*A)
display(Y_t, r_t, w_t, K_t1, H_t1, L_t1, A_t1)

Eq(Y_t, H_t**phi*K_t**alpha*(A_t*L_t)**(-alpha - phi + 1))

Eq(r_t, alpha*(H_t/(A_t*L_t))**phi*(K_t/(A_t*L_t))**(alpha - 1))

Eq(w_t, A_t*(H_t/(A_t*L_t))**phi*(K_t/(A_t*L_t))**alpha*(1 - alpha))

Eq(K_{t+1}, K_t*(1 - delta) + Y_t*s_K)

Eq(H_{t+1}, H_t*(1 - delta) + Y_t*s_H)

Eq(L_{t+1}, L_t*(n + 1))

Eq(A_{t+1}, A_t*(g + 1))

The Solow model with human capital is given with these equations. 1) stating that output is given as a Cobb-Douglas production function. 2 and 3) stating that the real rental rate on capital and real wage is given from the marginalproduct of capital and labour respectively. 4 and 5) stating the accumulation equation of capital and human capital. 6) stating that labour grows with the exogeneously rate of n. 7) stating that technology grows with the exogeneously rate of g.

# Solving the model analytically (symbolicaly) 

First we would like to solve the model symbolically with sympy meaning that we find the steady state values to thereafter solve for different values of steady state. We begin by defining the Solow eqautions.

In [13]:
#First we define the numerous new symbols with sympy
h_tilde = sm.symbols('htilde_t')       #Human capital per effective worker
h_tilde1 = sm.symbols('htilde_{t+1}')      #Human capital per worker in the next period
k_tilde = sm.symbols('ktilde_t')       #Capital per effective worker
k_tilde1 = sm.symbols('ktilde_{t+1}')  #Capital per effective worker in next period

For simplicity and time consumption we state the Solow equations down under. (Sympy cannot have operand types of - and so on on the left hand side, so therefore it is written in Latex form)

\begin{equation}
\tilde{k}_{t+1} - \tilde{k}_t = \frac{1}{(1+n)(1+g)}(s_K \tilde{k}_t^{\alpha} \tilde{h}_t^{\phi} - (n+g+\delta +ng)\tilde{k}_t)
\end{equation}
\begin{equation}
\tilde{h}_{t+1} - \tilde{h}_t = \frac{1}{(1+n)(1+g)}(s_H \tilde{k}_t^{\alpha} \tilde{h}_t^{\phi} - (n+g+\delta +ng)\tilde{h}_t)
\end{equation}

As we know analytically these equations can be solved to find steady state by setting k_tilde{t+1} and k_tilde equal to eachother and solving for h_tilde. That is the same as solving the right hand side and the left hand side in the big parenthesis should be equal to eachother.

In [14]:
#Here we create the equations to be solved
h_tilde_phi = h_tilde**phi
k_tilde_alpha = k_tilde**alpha
k = sm.Eq(sk * k_tilde_alpha * h_tilde_phi - (n + g + delta + n * g) * k_tilde, 0)
h = sm.Eq(sh * k_tilde_alpha * h_tilde_phi - (n + g+ delta + n * g) * h_tilde, 0)
display(k, h)

Eq(htilde_t**phi*ktilde_t**alpha*s_K - ktilde_t*(delta + g*n + g + n), 0)

Eq(-htilde_t*(delta + g*n + g + n) + htilde_t**phi*ktilde_t**alpha*s_H, 0)

Now we can solve this for k_tilde and h_tilde respectively by isolating k_tilde in the first eqaution and h_tilde in the second equation.

In [15]:
k_solve = sm.solve(k, k_tilde)[0]
h_solve = sm.solve(h, h_tilde)[0]
display(k_solve, h_solve)

((delta + g*n + g + n)/(htilde_t**phi*s_K))**(1/(alpha - 1))

((delta + g*n + g + n)/(ktilde_t**alpha*s_H))**(1/(phi - 1))

Now that we know what k_tilde and h_tilde is equal to we insert this in the k and h equations respectively.

In [16]:
#For k_tilde we inset the above equation onto k_tilde place in the k eqaution
k_solve_alpha = k_solve**alpha
k_inset = sm.Eq(sk * k_solve_alpha * h_tilde_phi - (n + g + delta + n * g) * k_solve, 0)
#For h_tilde we do the same. So inset it in the h equation
h_solve_phi = h_solve**phi
h_inset = sm.Eq(sh * k_tilde_alpha * h_solve_phi - (n + g+ delta + n * g) * h_solve, 0)
display(k_inset, h_inset)

Eq(htilde_t**phi*s_K*(((delta + g*n + g + n)/(htilde_t**phi*s_K))**(1/(alpha - 1)))**alpha - ((delta + g*n + g + n)/(htilde_t**phi*s_K))**(1/(alpha - 1))*(delta + g*n + g + n), 0)

Eq(ktilde_t**alpha*s_H*(((delta + g*n + g + n)/(ktilde_t**alpha*s_H))**(1/(phi - 1)))**phi - ((delta + g*n + g + n)/(ktilde_t**alpha*s_H))**(1/(phi - 1))*(delta + g*n + g + n), 0)

Now all that is left to do is to solve for h_tilde and k_tilde respectively and we get the steady state value for h_tilde and k_tilde.

In [18]:
k_ss = sm.solve(k_inset, k_tilde)[0]
h_ss = sm.solve(h_inset, k_tilde)[0]
display(k_ss, h_ss)

NotImplementedError: multiple generators [((delta/(htilde_t**phi*s_K) + g*n/(htilde_t**phi*s_K) + g/(htilde_t**phi*s_K) + n/(htilde_t**phi*s_K))**(1/(alpha - 1)))**alpha, (delta/(htilde_t**phi*s_K) + g*n/(htilde_t**phi*s_K) + g/(htilde_t**phi*s_K) + n/(htilde_t**phi*s_K))**(1/(alpha - 1)), htilde_t**phi]
No algorithms are implemented to solve equation htilde_t**phi*s_K*(((delta + g*n + g + n)/(htilde_t**phi*s_K))**(1/(alpha - 1)))**alpha - ((delta + g*n + g + n)/(htilde_t**phi*s_K))**(1/(alpha - 1))*(delta + g*n + g + n) + 0

So we get our steady state values of k_tilde and h_tilde to be:

In [24]:
#print('h_tilde^* is {hss})
#print('k_tilde^* is {kss})

# Interactive phase diagram

# Solving the model numerically